# CUDA C/C++ - A Beginner's Guide

This notebook teaches GPU programming with CUDA, explained for programmers familiar with C.

Inspired from Mark Harris's [An Even Easier Introduction to CUDA](https://developer.nvidia.com/blog/even-easier-introduction-cuda/).

**What you'll learn:**
- How CPU and GPU work together
- Writing and running GPU kernels
- Parallel programming with threads and blocks
- Profiling GPU performance

---
## Prerequisites

### Requirements

- CUDA-capable GPU (any NVIDIA GPU)
- C++ compiler (g++)
- [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit)
- Python environment for Jupyter

### Install Miniconda

Miniconda provides Python and the conda package manager. We need it because:
- Jupyter notebooks require a Python kernel to execute cells
- VS Code's Jupyter extension connects to conda environments

In [ ]:
%%bash
if [ ! -d "$HOME/miniconda3" ]; then
    wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/miniconda3
    $HOME/miniconda3/bin/conda init bash
    echo "Miniconda installed. Run: source ~/.bashrc"
else
    echo "Miniconda already installed"
fi

### Install IPython Kernel

The IPython kernel (`ipykernel`) bridges Jupyter and Python - required to run notebook cells.

In [ ]:
%%bash
if ! conda list -n base ipykernel 2>/dev/null | grep -q ipykernel; then
    conda install -n base ipykernel --update-deps --force-reinstall -y
else
    echo "ipykernel already installed"
fi

### Install CUDA Toolkit (Ubuntu 24.04)

The CUDA Toolkit provides:
- `nvcc` - the NVIDIA CUDA compiler for `.cu` files
- CUDA runtime libraries
- Header files and APIs
- Profiling tools (Nsight Systems, Nsight Compute)

In [ ]:
%%bash
if ! dpkg -l cuda-toolkit-13-1 2>/dev/null | grep -q ^ii; then
    wget -nc https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb
    sudo dpkg -i cuda-keyring_1.1-1_all.deb
    sudo apt-get update
    sudo apt-get -y install cuda-toolkit-13-1
else
    echo "cuda-toolkit-13-1 already installed"
fi

### Add CUDA to PATH

CUDA binaries are installed but not in your shell's PATH:

In [ ]:
%%bash
if ! grep -q 'cuda' ~/.bashrc; then
    echo 'export PATH=/usr/local/cuda/bin:$PATH' >> ~/.bashrc
    echo 'export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH' >> ~/.bashrc
    echo "Added CUDA to PATH. Run: source ~/.bashrc"
else
    echo "CUDA PATH already configured"
fi

### Install NVIDIA Driver

The driver enables OS-to-GPU communication. Choose ONE option:

In [ ]:
%%bash
# Option 1: Open source driver
if ! dpkg -l nvidia-open 2>/dev/null | grep -q ^ii; then
    sudo apt-get install -y nvidia-open
else
    echo "nvidia-open already installed"
fi

### Tip
For your g4dn.xlarge (the instance that I'm using) with NVIDIA T4 GPU, nvidia-open (the open-source kernel modules) is recommeded.

Reasons:
- NVIDIA officially recommends open kernel modules for datacenter GPUs (Turing and newer, which includes T4)
- Better integration with the Linux kernel
- Required for some newer features
- cuda-drivers installs the proprietary modules which are now considered legacy for datacenter cards

In [ ]:
%%bash
# Option 2: Proprietary driver (use this OR the above, not both)
if ! dpkg -l cuda-drivers 2>/dev/null | grep -q ^ii; then
    # sudo apt-get install -y cuda-drivers
    echo "All good. Open Source drivers are installed instead."
else
    echo "cuda-drivers already installed. Let's remove them, because open source drives are alerady installed"
    sudo apt-get remove -y cuda-drivers

fi

### Verify Installation

In [1]:
%%bash
command -v g++ >/dev/null && echo "✓ g++ installed" || echo "✗ g++ not found"
/usr/local/cuda/bin/nvcc --version >/dev/null 2>&1 && echo "✓ nvcc installed" || echo "✗ nvcc not found"
command -v nvidia-smi >/dev/null && echo "✓ nvidia-smi installed" || echo "✗ nvidia-smi not found"

✓ g++ installed
✓ nvcc installed
✓ nvidia-smi installed


---
## Part 1: Refresher

CUDA uses C++, but you only need a few differences from ANSI C:

### Output: `iostream` vs `stdio.h`

| C | C++ |
|---|---|
| `#include <stdio.h>` | `#include <iostream>` |
| `printf("x = %d\n", x);` | `std::cout << "x = " << x << std::endl;` |

**Good news:** `printf()` still works in CUDA! Use whichever you prefer.

### Memory: `new/delete` vs `malloc/free`

| C | C++ |
|---|---|
| `float *x = malloc(N * sizeof(float));` | `float *x = new float[N];` |
| `free(x);` | `delete[] x;` |

In CUDA, we'll use `cudaMallocManaged()` instead of both.

### Bit Shift Notation: `1<<20`

Same as C! It means 2²⁰ = 1,048,576 (about 1 million).

Another example: 
1<<3 = 1 × 2³
2<<3 = 2 × 2³

In [2]:
%%bash
echo "1<<20 = $((1<<20))"

1<<20 = 1048576


---
## Part 2: CPU vs GPU - The Mental Model

| | CPU | GPU |
|---|---|---|
| **Cores** | 4-16 fast cores | 1000s of slower cores |
| **Good at** | Complex tasks, one at a time | Simple tasks, many at once |
| **Memory** | RAM (host memory) | VRAM (device memory) |
| **Code name** | Host code | Device code / Kernel |

**Key insight:** GPUs are fast because they do the SAME operation on MANY data points simultaneously.

**VRAM** (Video RAM) is the dedicated memory on a GPU.

It's separate from your system RAM and sits physically on the graphics card, connected directly to the GPU cores via a high-bandwidth bus.

Key characteristics:
- Much faster than system RAM (hundreds of GB/s vs tens of GB/s)
- Limited capacity (8-80 GB on modern GPUs vs 64-512 GB system RAM)
- Data must be copied from system RAM to VRAM before the GPU can process it

### My GPU:

In [3]:
%%bash
nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

# SMI = System Management Interface

name, memory.total [MiB], compute_cap
Tesla T4, 15360 MiB, 7.5


**Example: g4dn.xlarge with Tesla T4**

| Field | Value | Meaning |
|-------|-------|---------|
| name | Tesla T4 | Turing architecture, inference-optimized |
| memory.total | 15360 MiB | ~15 GB VRAM |
| compute_cap | 7.5 | Compile with `-arch=sm_75` |

**Compute Capability** indicates the GPU's architecture generation and supported CUDA features. The T4's 7.5 means Turing architecture. The `sm` in `-arch=sm_75` stands for **Streaming Multiprocessor** - the core processing unit on NVIDIA GPUs. Always match this when compiling:

```bash
nvcc -arch=sm_75 program.cu -o program
```

Using the wrong architecture (e.g., `sm_80` for Ampere) will compile successfully but fail silently at runtime.

**Why precision matters for T4:**

The T4 delivers 8.1 TFLOPS at FP32 but jumps to 65 TFLOPS at FP16 - that's 8x faster! Understanding precision helps you choose the right format:

| Format | Name | Bits | Exponent | Mantissa | Range | Precision |
|--------|------|------|----------|----------|-------|-----------|
| FP32 | Single precision | 32 | 8 | 23 | ±~3.4 × 10³⁸ | ~7-8 digits |
| FP16 | Half precision | 16 | 5 | 10 | ±~6.5 × 10⁴ | ~3-4 digits |

FP32 is critical for training where gradient accuracy matters. FP16 / INT8 is sufficient for inference since small rounding errors don't affect predictions. The T4 is optimized for the latter, making it ideal for deploying trained models.

**Note:** Precision isn't set in the `nvcc` command - `-arch=sm_75` only targets the GPU architecture. Precision is controlled in your code by using `float` (FP32) or `half`/`__half` (FP16) data types, or via libraries like cuBLAS with precision options.

**T4 vs Training GPUs:**

| GPU | FP32 TFLOPS | Use Case |
|-----|-------------|----------|
| T4 | 8.1 | Inference, learning CUDA |
| V100 | 15.7 | Training |
| A100 | 19.5 | Training |
| H100 | 67 | Large-scale training |
| H200 | 67 | Large models (141 GB HBM3e) |

The T4 excels at FP16/INT8 inference but has lower FP32 throughput than training GPUs. This makes it ideal for deploying trained models in production or learning CUDA at significantly lower cost.

### Your GPU specs:

In [ ]:
%%bash
nvidia-smi

---
## Part 3: Starting Simple - Array Addition

We'll add two arrays element by element:
```
x = [1, 1, 1, ...] (1 billion elements)
y = [2, 2, 2, ...] (1 billion elements)
result: y = [3, 3, 3, ...]
```

First, let's do it on the CPU (pure C - this should look familiar):

In [4]:
%%writefile add.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// Function to add elements of two arrays
void add(int n, float *x, float *y) {
    for (int i = 0; i < n; i++)
        y[i] = x[i] + y[i];
}

int main() {
    int N = 1<<30;  // 1 billion elements
    
    float *x = malloc(N * sizeof(float));
    float *y = malloc(N * sizeof(float));
    
    // Initialize: x=1, y=2
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }
    
    add(N, x, y);  // Do the addition
    
    // Check for errors (all values should be 3.0)
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(y[i] - 3.0f));
    printf("Max error: %f\n", maxError);
    
    free(x);
    free(y);
    return 0;
}

Overwriting add.c


Compile and run:

In [5]:
%%bash
gcc add.c -o add -lm && ./add

Max error: 0.000000


**The opportunity:** This loop runs 1 billion iterations sequentially - it works, but each addition waits for the previous one (~17 seconds on this machine's CPU). What if we could do all additions AT THE SAME TIME?

That's where the GPU comes in.

---
## Part 4: Your First CUDA Program

To run code on the GPU, we need **3 changes**:

### Change 1: Mark the function with `__global__`

**CPU version:**
```c
void add(int n, float *x, float *y) { ... }
```

**GPU version - add `__global__`:**
```c
__global__ void add(int n, float *x, float *y) { ... }
```

`__global__` tells the compiler: "This function runs on the GPU but is called from the CPU."

These functions are called **kernels**. Code on GPU = **device code**, code on CPU = **host code**.

**Important:** `__global__` is a **function qualifier** - the entire function body (everything between `{` and `}`) runs on GPU:

```c
__global__
void add(int n, float *x, float *y) {   // Scope starts
    for (int i = 0; i < n; i++)         // GPU code
        y[i] = x[i] + y[i];             // GPU code
}                                        // Scope ends
```

You cannot mark a block of code inside a function to run on GPU - you must create a separate `__global__` function and call it with `<<<blocks, threads>>>`.

**CUDA function qualifiers:**

| Qualifier | Runs on | Called from | Use case |
|-----------|---------|-------------|----------|
| `__global__` | GPU | CPU (or GPU with dynamic parallelism) | Kernel entry points |
| `__device__` | GPU | GPU only | Helper functions called by kernels |
| `__host__` | CPU | CPU | Regular CPU functions (default, optional) |

**Combining qualifiers:**
```c
__host__ __device__ float square(float x) { return x * x; }
```
This compiles the function for both CPU and GPU - useful for utility functions you need in both places.

### Change 2: Use Unified Memory

```c
// CPU only
float *x = malloc(N * sizeof(float));
free(x);

// CPU + GPU (Unified Memory)
float *x;
cudaMallocManaged(&x, N * sizeof(float));
cudaFree(x);
```

[Unified Memory](https://developer.nvidia.com/blog/unified-memory-in-cuda-6/) creates memory accessible by both CPU and GPU automatically.

**How it works:**

Despite the name, Unified Memory isn't truly "shared" - data physically copies between CPU RAM and GPU VRAM. What's unified is the **address space** (single pointer works everywhere), not the memory itself:

1. CPU writes to array → data lives in CPU RAM
2. GPU kernel launches → data copies to GPU VRAM over PCIe
3. CPU reads results → data copies back to CPU RAM

**What is an address space?**

An address space is the range of memory addresses a processor can use. Think of it like street addresses - CPU has addresses 0x0000-0xFFFF for its RAM, GPU has its own addresses for VRAM. Normally these are separate: a pointer valid on CPU means nothing to the GPU.

Unified Memory creates a **single virtual address space** where one pointer (e.g., `0x7f3a...`) is valid on both. The CUDA runtime translates this to the actual physical location and copies data as needed.

**Note:** CPU and GPU don't need the same amount of memory. The address space is virtual - you can allocate more than your GPU's VRAM (15 GB on T4). Overflow stays in CPU RAM and pages into VRAM as needed. On multi-GPU systems, overflow still goes to CPU RAM (not other GPUs) - the benefit is simpler programming, not combined VRAM capacity.

**Why true sharing is impossible:**

CPU RAM sits on the motherboard, connected to the CPU. GPU VRAM sits on the graphics card, connected to the GPU. They're physically separate chips connected only by PCIe (a relatively slow bus). For true sharing, both processors would need to access the same memory chips - which would require them to be on the same silicon or share a memory bus. Some APUs (Accelerated Processing Units - chips combining CPU and GPU on the same die, like PlayStation 5 or AMD Ryzen with integrated graphics) do this, but discrete GPUs like your T4 cannot.

This automatic migration has overhead. For production code, explicit memory management gives more control and better performance:

**Step 1: Allocate memory on both CPU and GPU**
```c
float *h_x = malloc(N * sizeof(float));  // Host (CPU) memory
float *d_x;                               // Device (GPU) pointer
cudaMalloc(&d_x, N * sizeof(float));     // Allocate on GPU
```

**Step 2: Initialize data on CPU**
```c
for (int i = 0; i < N; i++) h_x[i] = 1.0f;
```

**Step 3: Copy data from CPU to GPU**
```c
cudaMemcpy(d_x, h_x, N * sizeof(float), cudaMemcpyHostToDevice);
```

**Step 4: Run kernel on GPU**
```c
kernel<<<blocks, threads>>>(d_x);
```

**Step 5: Copy results back from GPU to CPU**
```c
cudaMemcpy(h_x, d_x, N * sizeof(float), cudaMemcpyDeviceToHost);
```

**Step 6: Free memory**
```c
cudaFree(d_x);  // Free GPU memory
free(h_x);      // Free CPU memory
```

This is faster because you control exactly when copies happen, and can overlap computation with data transfer.

**When to use which:**
- **Unified Memory:** Simpler code, good for learning, prototyping, unpredictable access patterns, datasets larger than VRAM, or multi-GPU systems (runtime automatically migrates data between GPUs - no manual `cudaMemcpyPeer` or tracking which data is where)
- **Explicit (`cudaMalloc` + `cudaMemcpy`):** Maximum performance when you know exactly when data needs to move

**Caution on multi-GPU:** If multiple GPUs frequently access the same data, it keeps copying back and forth ("thrashing"), hurting performance. With NVLink (high-end systems), copies go directly between GPUs. Without NVLink (most systems, including g4dn), copies route through CPU RAM (GPU0 → RAM → GPU1), which is slower. Explicit management lets you control which data stays on which GPU.

### Change 3: Launch with `<<<blocks, threads>>>`

```c
// CPU call
add(N, x, y);

// GPU call
add<<<1, 1>>>(N, x, y);      // Launch kernel
cudaDeviceSynchronize();      // Wait for GPU to finish
```

**Anatomy of a kernel launch:**

```
add<<<1, 1>>>(N, x, y);
│   │  │   │  └─────── Function arguments (same as CPU)
│   │  │   └────────── Threads per block
│   │  └────────────── Number of blocks
│   └───────────────── Execution configuration (CUDA-specific)
└───────────────────── Kernel function name
```

The `<<<blocks, threads>>>` syntax is CUDA's way of specifying parallelism:
- `<<<1, 1>>>` = 1 block × 1 thread = 1 total thread (sequential, slow)
- `<<<1, 256>>>` = 1 block × 256 threads = 256 parallel threads
- `<<<4096, 256>>>` = 4096 blocks × 256 threads = ~1 million parallel threads

`cudaDeviceSynchronize()` blocks the CPU until the GPU finishes - necessary because kernel launches are asynchronous (CPU continues immediately without waiting).

**Why synchronization matters:**

```c
add<<<blocks, threads>>>(N, x, y);  // CPU sends work to GPU and continues immediately
printf("%f", y[0]);                 // BUG: GPU might not be done yet!

// Correct:
add<<<blocks, threads>>>(N, x, y);  // CPU sends work to GPU
cudaDeviceSynchronize();             // CPU waits here until GPU finishes
printf("%f", y[0]);                 // Safe: GPU is definitely done
```

Without synchronization, the CPU might read results before the GPU has written them - leading to incorrect or garbage values.

**Note:** `cudaDeviceSynchronize()` blocks the CPU thread - those cycles are wasted waiting. If you have CPU work that doesn't depend on GPU results, do it before syncing:

```c
kernel<<<blocks, threads>>>(d_x);  // GPU starts, CPU continues immediately
prepare_next_batch();              // CPU work that doesn't need GPU results
write_logs();                      // More independent work
cudaDeviceSynchronize();           // NOW wait for GPU
use_results(d_x);                  // Safe to use GPU results
```

If your CPU work depends on GPU output, you must wait - this optimization only helps when you have independent work.

Advanced: CUDA streams and events allow even finer control over async operations - but that's beyond this beginner's guide.

### CUDA code in action:

In [6]:
%%writefile add.cu
#include <stdio.h>
#include <math.h>

// Kernel function - runs on GPU
__global__
void add(int n, float *x, float *y) {
    for (int i = 0; i < n; i++)
        y[i] = x[i] + y[i];
}

int main() {
    int N = 1<<30;  // 1 billion elements
    float *x, *y;
    
    // Allocate Unified Memory - accessible from CPU or GPU
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));
    
    // Initialize on CPU (simple, but slow for large arrays. (We will improve it later)
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }
    
    // Run kernel on GPU (1 block, 1 thread - intentionally slow!)
    add<<<1, 1>>>(N, x, y);
    
    // Wait for GPU to finish before accessing results
    cudaDeviceSynchronize();
    
    // Check for errors (using CPU, for now)
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
       maxError = fmax(maxError, fabs(y[i] - 3.0f));
    printf("Max error: %f\n", maxError);
    
    cudaFree(x);
    cudaFree(y);
    return 0;
}

Overwriting add.cu


Compile with `nvcc` (NVIDIA CUDA Compiler) and run:

**What is nvcc?** NVIDIA's compiler for CUDA code. It separates CPU code from GPU code, compiles GPU code to machine instructions for your GPU, passes CPU code to `g++`, and links everything into one executable.

In [7]:
%%bash
/usr/local/cuda/bin/nvcc add.cu -o add_cuda && ./add_cuda

Max error: 0.000000


**It works!** But this is actually much SLOWER than CPU, because we're only using 1 GPU thread.

You might think: "Just change `<<<1, 1>>>` to `<<<1, 256>>>` for more threads!" But that won't help with this kernel. Look at its loop:

```c
for (int i = 0; i < n; i++)  // Every thread would run this same loop!
```

With `<<<1, 256>>>`, all 256 threads would execute the **same loop** from 0 to N:
- Thread 0: processes elements 0, 1, 2, ... N-1
- Thread 1: processes elements 0, 1, 2, ... N-1
- Thread 2: processes elements 0, 1, 2, ... N-1
- (all 256 threads do identical work)

That's 256x the work for no benefit! We need to rewrite the kernel so each thread handles **different** elements using `threadIdx.x`.

---
## Part 5: Profiling with Nsight Systems

**Nsight Systems** is NVIDIA's system-wide profiler that shows CPU/GPU activity, memory transfers, and kernel timings on a unified timeline. The `nsys` command is its CLI tool.

Let's measure how long the kernel takes:

```bash
nsys profile --stats=true ./add_cuda
```

This generates two files:
- **`.nsys-rep`** - Native report format for the Nsight Systems GUI (`nsys-ui`)
- **`.sqlite`** - SQLite database for programmatic queries

The `.sqlite` file is created automatically when you use `--stats=true` because nsys computes statistics via SQL queries internally. You can also explicitly request it with `--export=sqlite`.

In [8]:
%%bash
nsys profile --stats=true ./add_cuda 2>&1 | grep -A 10 'cuda_gpu_kern_sum'

[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances    Avg (ns)       Med (ns)      Min (ns)     Max (ns)    StdDev (ns)             Name           
 --------  ---------------  ---------  -------------  -------------  -----------  -----------  -----------  --------------------------
    100.0      45828070907          1  45828070907.0  45828070907.0  45828070907  45828070907          0.0  add(int, float *, float *)

[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)               Operation              
 --------  ---------------  -----  --------  --------  --------  --------  -----------  ------------------------------------
     68.3       1449746401  49152   29495.2   10223.0      1855    169789      48194.8  [CUDA memcpy Unified Host-to-Device]


**Reading nsys output:**

The `cuda_gpu_kern_sum` section shows **only GPU kernel execution time**, not total program time:

| Column | Meaning |
|--------|--------|
| Time (%) | Percentage of total GPU time spent in this kernel |
| Total Time (ns) | Kernel execution time in nanoseconds (divide by 1,000,000,000 for seconds) |
| Instances | How many times the kernel was called |
| Name | Kernel function name with parameters |

**Why total program time is longer than kernel time:**

The program does more than just run the kernel:
1. CPU initialization loop (1 billion iterations) - ~15-20 seconds
2. GPU kernel execution - shown in nsys output
3. CPU error-checking loop - a few seconds

**Breaking down the numbers:**

The nsys output shows `Total Time (ns)` for the kernel. To convert nanoseconds to seconds, divide by 1,000,000,000 (or 10⁹). For example, `45814840481 ns ÷ 10⁹ = ~45.8 seconds`.

If you time the entire program (e.g., with `time ./add_cuda`), you'll see it takes longer than the kernel time alone. The difference is CPU work: initialization loops, error checking, and CUDA setup overhead.

With 1 billion elements and `<<<1, 1>>>`, the `add` kernel will be very slow. Let's make it faster.

---
## Part 6: Understanding GPU Threads

### Thread Organization

Threads are grouped into **blocks**, and blocks form a **grid**:

<img src="https://developer-blogs.nvidia.com/wp-content/uploads/2017/01/cuda_indexing.png" width="700">

Why two levels? Threads in the same block can share fast memory and synchronize. Threads across blocks cannot."

### How Threads Find Their Work

Each thread has built-in variables to identify itself:

| Variable | What it tells you |
|----------|-------------------|
| `threadIdx.x` | \"I'm thread #5 in my block\" |
| `blockIdx.x` | \"I'm in block #2\" |
| `blockDim.x` | \"My block has 256 threads\" |

**The Key Formula:**
```c
int i = blockIdx.x * blockDim.x + threadIdx.x;
```

**Example:** If you're thread 5 in block 2, and blocks have 256 threads:
```
i = 2 * 256 + 5 = 517
```
So you process `array[517]`."

### Why `.x`?

CUDA supports 1D, 2D, and 3D layouts. The `.x` selects the dimension:

| Layout | Use case | Variables |
|--------|----------|----------|
| 1D | Arrays | `.x` |
| 2D | Images | `.x`, `.y` |
| 3D | Volumes | `.x`, `.y`, `.z` |

For arrays, 1D is enough - we only use `.x`."

### Let's see thread IDs in action:

In [9]:
%%writefile show_threads.cu
#include <stdio.h>

__global__ void showThreads() {
    int globalId = blockIdx.x * blockDim.x + threadIdx.x;
    printf("Block %d, Thread %d -> Global ID: %d\n",
           blockIdx.x, threadIdx.x, globalId);
}

int main() {
    printf("Launching 2 blocks x 4 threads = 8 threads total:\n\n");
    showThreads<<<2, 4>>>();
    cudaDeviceSynchronize();
    return 0;
}

Overwriting show_threads.cu


In [10]:
%%bash
/usr/local/cuda/bin/nvcc show_threads.cu -o show_threads && ./show_threads

Launching 2 blocks x 4 threads = 8 threads total:

Block 1, Thread 0 -> Global ID: 4
Block 1, Thread 1 -> Global ID: 5
Block 1, Thread 2 -> Global ID: 6
Block 1, Thread 3 -> Global ID: 7
Block 0, Thread 0 -> Global ID: 0
Block 0, Thread 1 -> Global ID: 1
Block 0, Thread 2 -> Global ID: 2
Block 0, Thread 3 -> Global ID: 3


**Notice:** Output order is random! Threads run in parallel, not sequentially.

---
## Part 7: Parallelizing with One Block

Let's use 256 threads in one block. Each thread handles a portion of the array:

```
Thread 0: processes elements 0, 256, 512, ...
Thread 1: processes elements 1, 257, 513, ...
Thread 2: processes elements 2, 258, 514, ...
```

This is called a **stride loop** - each thread strides through the array.

### The kernel with stride:

```c
__global__
void add(int n, float *x, float *y) {
    int index = threadIdx.x;      // Starting position
    int stride = blockDim.x;      // Step size (256)
    for (int i = index; i < n; i += stride)
        y[i] = x[i] + y[i];
}
```

**Why `stride = blockDim.x`?** We have 256 threads, each needs a unique starting point (`index`), and they must not overlap. By stepping by the total thread count, each element is processed exactly once:

```
Array:    [0] [1] [2] ... [255] [256] [257] ... [511] [512] ...
Thread 0:  ^                      ^                     ^
Thread 1:      ^                        ^                     ^
Thread 2:          ^                          ^
```

If stride were smaller (say 128), threads 0 and 128 would both try to process element 256 - causing duplicate work or race conditions.

In [4]:
%%writefile add_block.cu
#include <stdio.h>
#include <math.h>

__global__
void add(int n, float *x, float *y) {
    int index = threadIdx.x;      // This thread's starting index
    int stride = blockDim.x;      // Total threads = step size
    for (int i = index; i < n; i += stride)
        y[i] = x[i] + y[i];
}

int main() {
    int N = 1<<30;  // 1 billion elements
    float *x, *y;
    
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));
    
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }
    
    // 1 block, 256 threads (better, but still limited)
    add<<<1, 256>>>(N, x, y);
    cudaDeviceSynchronize();
    
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(y[i] - 3.0f));
    printf("Max error: %f\n", maxError);
    
    cudaFree(x);
    cudaFree(y);
    return 0;
}

Overwriting add_block.cu


In [5]:
%%bash
/usr/local/cuda/bin/nvcc add_block.cu -o add_block
nsys profile --stats=true ./add_block 2>&1 | grep -A 10 'cuda_gpu_kern_sum'

[6/8] Executing 'cuda_gpu_kern_sum' stats report

 Time (%)  Total Time (ns)  Instances    Avg (ns)      Med (ns)     Min (ns)    Max (ns)   StdDev (ns)             Name           
 --------  ---------------  ---------  ------------  ------------  ----------  ----------  -----------  --------------------------
    100.0       3466668059          1  3466668059.0  3466668059.0  3466668059  3466668059          0.0  add(int, float *, float *)

[7/8] Executing 'cuda_gpu_mem_time_sum' stats report

 Time (%)  Total Time (ns)  Count  Avg (ns)  Med (ns)  Min (ns)  Max (ns)  StdDev (ns)               Operation              
 --------  ---------------  -----  --------  --------  --------  --------  -----------  ------------------------------------
     68.3       1450967669  49152   29520.0    7151.5      1855    169629      48171.5  [CUDA memcpy Unified Host-to-Device]


The Total Time (ns) in above example is roughly 3 seconds. 

**Big speedup!** We went from 1 thread to 256. But GPUs have thousands of cores - let's use more.

---
## Part 8: Using Multiple Blocks

GPUs have many **Streaming Multiprocessors (SMs)**, each running multiple thread blocks. For example, a Tesla P100 has 56 SMs, each supporting up to 2048 threads.

To use all this power, we launch **multiple blocks**:

```c
int blockSize = 256;
int numBlocks = (N + blockSize - 1) / blockSize;  // Round up
add<<<numBlocks, blockSize>>>(N, x, y);
```

The kernel uses a [grid-stride loop](https://developer.nvidia.com/blog/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/) to handle arrays larger than the total thread count:

```c
__global__
void add(int n, float *x, float *y) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;  // Total threads in grid
    for (int i = index; i < n; i += stride)
        y[i] = x[i] + y[i];
}
```

In [40]:
%%writefile add_grid.cu
#include <stdio.h>
#include <math.h>

__global__
void init(int n, float *x, float *y) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (; i < n; i += stride) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }
}

__global__
void add(int n, float *x, float *y) {
    // Global thread index
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    // Total threads in entire grid
    int stride = blockDim.x * gridDim.x;
    
    for (int i = index; i < n; i += stride)
        y[i] = x[i] + y[i];
}

int main() {
    int N = 1<<30;  // 1 billion elements
    float *x, *y;
    
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));
    
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;
    
    init<<<numBlocks, blockSize>>>(N, x, y);
    
    printf("Launching %d blocks x %d threads\n", numBlocks, blockSize);
    add<<<numBlocks, blockSize>>>(N, x, y);
    cudaDeviceSynchronize();
    
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(y[i] - 3.0f));
    printf("Max error: %f\n", maxError);
    
    cudaFree(x);
    cudaFree(y);
    return 0;
}

Overwriting add_grid.cu


In [41]:
%%bash
/usr/local/cuda/bin/nvcc add_grid.cu -o add_grid
# nsys profile --stats=true ./add_grid 2>&1 | grep -A 10 'cuda_gpu_kern_sum'
./add_grid

Launching 4194304 blocks x 256 threads
Max error: 0.000000


**Another speedup!** We're now using the full power of the GPU.

---
## Summary: CPU to CUDA Cheat Sheet

| What | CPU (C) | GPU (CUDA) |
|------|---------|------------|
| Function | `void func()` | `__global__ void func()` |
| Allocate | `malloc(size)` | `cudaMallocManaged(&ptr, size)` |
| Free | `free(ptr)` | `cudaFree(ptr)` |
| Call | `func(args)` | `func<<<blocks, threads>>>(args)` |
| Wait | (automatic) | `cudaDeviceSynchronize()` |
| Thread ID | N/A | `blockIdx.x * blockDim.x + threadIdx.x` |
| File | `.c` | `.cu` |
| Compiler | `gcc` | `nvcc` |

---
## Resources

**Documentation:**
- [CUDA C++ Programming Guide](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html)
- [CUDA Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html)
- [CUDA Toolkit Documentation](https://docs.nvidia.com/cuda/index.html)

**Courses:**
- [Fundamentals of Accelerated Computing with CUDA C/C++](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-01+V1/about) - NVIDIA DLI
- [Fundamentals of Accelerated Computing with CUDA Python](https://courses.nvidia.com/courses/course-v1:DLI+C-AC-02+V1/about) - NVIDIA DLI

**Tools:**
- `nsys` - Nsight Systems command-line profiler (used in this notebook)
- [NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems) - Visual profiler
- [NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute) - Kernel profiler

---
*Based on Mark Harris's [An Even Easier Introduction to CUDA](https://developer.nvidia.com/blog/even-easier-introduction-cuda/)*